In [134]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, count, demographic_parity_difference
from ipynb.fs.defs import ERED

### Load Data

In [135]:
# Overall data
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
train_X = list(train_data['TFIDF'])
train_y = (train_data['Sentiment'] == 'positive') * 1

dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
dev_X = list(dev_data['TFIDF'])
dev_y  = (dev_data['Sentiment'] == 'positive') * 1

# AAE
# Filter and extract all AAE data
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
AAE_train_data = train_data[train_data['Demographic'] == 'AAE']
AAE_train_X = list(AAE_train_data['TFIDF'])
AAE_train_y = (AAE_train_data['Sentiment'] == 'positive') * 1

dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
AAE_dev_data = dev_data[dev_data['Demographic'] == 'AAE']
AAE_dev_X = list(AAE_dev_data['TFIDF'])
AAE_dev_y  = (AAE_dev_data['Sentiment'] == 'positive') * 1

# SAE
# Filter and extract all SAE data
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
SAE_train_data = train_data[train_data['Demographic'] == 'SAE']
SAE_train_X = list(SAE_train_data['TFIDF'])
SAE_train_y = (SAE_train_data['Sentiment'] == 'positive') * 1

dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
SAE_dev_data = dev_data[dev_data['Demographic'] == 'SAE']
SAE_dev_X = list(SAE_dev_data['TFIDF'])
SAE_dev_y  = (SAE_dev_data['Sentiment'] == 'positive') * 1

### Train Model

In [136]:
mlp = MLPClassifier(max_iter=500, random_state=0)
mlp.fit(train_X, train_y)

MLPClassifier(max_iter=500, random_state=0)

### Prediction

#### 1. Overall

In [137]:
overall_labels_predict = mlp.predict(dev_X)
overall_confusion = confusion_matrix(dev_y, overall_labels_predict)
print(classification_report(dev_y, overall_labels_predict))
print(overall_confusion)

              precision    recall  f1-score   support

           0       0.63      0.64      0.64      2000
           1       0.64      0.63      0.63      2000

    accuracy                           0.63      4000
   macro avg       0.63      0.63      0.63      4000
weighted avg       0.63      0.63      0.63      4000

[[1275  725]
 [ 736 1264]]


#### 2. AAE

In [138]:
AAE_labels_predict = mlp.predict(AAE_dev_X)
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(AAE_confusion)

              precision    recall  f1-score   support

           0       0.61      0.57      0.59      1000
           1       0.59      0.63      0.61      1000

    accuracy                           0.60      2000
   macro avg       0.60      0.60      0.60      2000
weighted avg       0.60      0.60      0.60      2000

[[571 429]
 [372 628]]


#### 3. SAE

In [139]:
SAE_labels_predict = mlp.predict(SAE_dev_X)
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(SAE_confusion)

              precision    recall  f1-score   support

           0       0.66      0.70      0.68      1000
           1       0.68      0.64      0.66      1000

    accuracy                           0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000

[[704 296]
 [364 636]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [140]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.63475
count          4000
dtype: object
            accuracy count
Demographic               
AAE           0.5995  2000
SAE             0.67  2000


#### 2. Selection rate + Demographic parity difference

In [141]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print("\nDifference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 3)))

0.49725
Demographic
AAE    0.5285
SAE     0.466
Name: selection_rate, dtype: object

Difference between selection rate
Demographic parity difference: 0.062


#### 3. Error Rate Equality Difference

In [142]:
matrix_lst = [AAE_confusion, SAE_confusion]
ERED.cal_sum_ered(overall_confusion, matrix_lst)

Sum of FPR difference: 0.133
Sum of FNR difference: 0.008
Sum of Error Rate Equality Difference: 0.141


0.14100000000000001